In [1]:
import pqr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf

# Выгрузка исходных данных для построения стратегии или модели

In [7]:
stocks = pd.read_csv("https://github.com/qununc/financials_repo/blob/main/data/stock_final_2012-2022.csv.xz?raw=true", compression="xz")

In [45]:
voluminous_stocks_tickers = stocks[stocks["Volume"] > 10 ** 8.825].loc[:, "Name"].unique()

In [46]:
with open("./hw_data/voluminous_stocks_tickers.json", "w"):
    dump

NameError: name 'dump' is not defined

# Обработка финансовых данных

# Построение инвестиционной стратегии или финансовой модели

NameError: name 'dust' is not defined

# Аналитика и визуализация результатов